<a href="https://colab.research.google.com/github/yours-schnee/software_retina/blob/main/software_retina_kai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/yours-schnee/RetinaVision.git

Cloning into 'RetinaVision'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 314 (delta 3), reused 0 (delta 0), pack-reused 306
Receiving objects: 100% (314/314), 224.17 MiB | 32.70 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Checking out files: 100% (64/64), done.


In [ ]:
!pip install -e ./RetinaVision

Obtaining file:///content/RetinaVision
  Running setup.py develop for retinavision


In [ ]:
import os
import cv2
import pickle
import sys
import glob
import os.path as osp

!alias pcat='python -m pickle'

In [ ]:
sys.path.append('/content/RetinaVision')

In [ ]:
import RetinaVision
from retinavision.retina import Retina
from retinavision.cortex import Cortex

In [ ]:
import numpy as np
from retinavision.cuda_objects import CudaRetina
from retinavision.utils import pad, loadPickle, project, loadPickleNonbin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pcat RetinaVision/data/retinas/8k_coeff.pkl

### Data load

In [ ]:
!cp "drive/My Drive/COCO/customCOCO/customTrainkai3.zip" /content/
!cp "drive/My Drive/COCO/customCOCO/customTestkai2.zip" /content/

In [ ]:
!unzip -nq "customTrainkai3.zip"
!unzip -nq "customTestkai2.zip"

In [ ]:
!cp "drive/My Drive/COCO/customCOCO/retina.zip" /content/
!cp "drive/My Drive/COCO/customCOCO/cortical.zip" /content/

In [ ]:
!unzip -nq "retina.zip"
!unzip -nq "cortical.zip"

### Retina & Cortical transform preparation

In [ ]:
shape = [224, 224]
classes = ['bg']

CONT_PKL = "8k"

In [ ]:
# Create and prepare retina
R = Retina()

if (CONT_PKL == "8k"):
  R.loadLoc('RetinaVision/data/retinas/8k_loc.pkl')
  R.loadCoeff('RetinaVision/data/retinas/8k_coeff.pkl')
else:
  R.loadLoc('RetinaVision/data/retinas/ret50k_loc.pkl')
  R.loadCoeff('RetinaVision/data/retinas/ret50k_coeff.pkl')

x = shape[1]//2
y = shape[0]//2
fixation = (y, x)
R.prepare(shape, fixation)

In [ ]:
# Create and prepare cortex
C = Cortex()

if (CONT_PKL == "8k"):
  C.loadLocs('RetinaVision/data/cortices/8k_cort_leftloc.pkl', 'RetinaVision/data/cortices/8k_cort_rightloc.pkl')
  C.loadCoeffs('RetinaVision/data/cortices/8k_cort_leftcoeff.pkl', 'RetinaVision/data/cortices/8k_cort_rightcoeff.pkl')
else:
  C.loadLocs('RetinaVision/data/cortices/50k_Lloc_tight.pkl', 'RetinaVision/data/cortices/50k_Rloc_tight.pkl')
  C.loadCoeffs('RetinaVision/data/cortices/50k_Lcoeff_tight.pkl', 'RetinaVision/data/cortices/50k_Rcoeff_tight.pkl')

In [ ]:
DEBUG = False
if DEBUG:
    classes = [ 'giraffe']
else:
    classes = [ 'giraffe', 'horse', 'motorcycle', 'sheep', 'stop sign', 'train', 'zebra',
            'airplane', 'banana', 'bear', 'bicycle', 'bird', 'boat', 'bus', 'cat',
            'cow', 'dog', 'donut', 'elephant', 'fire hydrant']

train_dir ='customTrainkai3/img/'
test_dir = 'customTestkai2/img/'

In [ ]:
# Creating png path list
def make_datapath_list(dir_name, class_name):
    target_dir = osp.join(dir_name+class_name+'/*.png')
    #print(target_dir)
    dir_list = []

    for dir in glob.glob(target_dir):
        dir_list.append(dir)

    return dir_list

In [ ]:
DEBUG2 = False

path_flg = True 
if True:
    path = train_dir
else:
    path = test_dir

In [ ]:
# Generate cortical images
for cla in classes:
    print('Processing {}'.format(cla))

    class_img_list = make_datapath_list(path, cla)
    
    length = 3 if DEBUG2 else len(class_img_list)
    for num in range(length):
        #print(class_img_list[num])
        img = cv2.imread(class_img_list[num])
        assert not isinstance(img,type(None)), 'image not found'

        retina_dir = '{}/img/{}/'.format('retina', cla)
        cortical_dir = '{}/img/{}/'.format('cortical', cla)

        #os.makedirs(cortical_dir, exist_ok=True)
        
        V = R.sample(img, fixation)
        tight = R.backproject_tight_last()
        cimg = C.cort_img(V)

        if DEBUG:
            cv2.imwrite('retina_{}_'.format(cla) + '{}.png'.format(num), tight)
            cv2.imwrite('cortical_{}_'.format(cla) + '{}.png'.format(num), cimg)
        elif num < 10:
            cv2.imwrite('{}/{}_00'.format(retina_dir, cla) + '{}.png'.format(num), tight)
            cv2.imwrite('{}/{}_00'.format(cortical_dir, cla) + '{}.png'.format(num), cimg)
        elif num < 100:
            cv2.imwrite('{}/{}_0'.format(retina_dir, cla) + '{}.png'.format(num), tight)
            cv2.imwrite('{}/{}_0'.format(cortical_dir, cla) + '{}.png'.format(num), cimg)
        else:
            cv2.imwrite('{}/{}_'.format(retina_dir, cla) + '{}.png'.format(num), tight)
            cv2.imwrite('{}/{}_'.format(cortical_dir, cla) + '{}.png'.format(num), cimg)
    print('{} finish'.format(cla))


Processing giraffe
giraffe finish
Processing horse
horse finish
Processing motorcycle
motorcycle finish
Processing sheep
sheep finish
Processing stop sign
stop sign finish
Processing train
train finish
Processing zebra
zebra finish
Processing airplane
airplane finish
Processing banana
banana finish
Processing bear
bear finish
Processing bicycle
bicycle finish
Processing bird
bird finish
Processing boat
boat finish
Processing bus
bus finish
Processing cat
cat finish
Processing cow
cow finish
Processing dog
dog finish
Processing donut
donut finish
Processing elephant
elephant finish
Processing fire hydrant
fire hydrant finish


In [ ]:
!zip -r /content/retina_train.zip /content/retina
!zip -r /content/cortical_train.zip /content/cortical

ストリーミング出力は最後の 5000 行に切り捨てられました。
  adding: content/cortical/img/giraffe/giraffe_324.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_137.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_347.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_361.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_181.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_379.png (deflated 2%)
  adding: content/cortical/img/giraffe/giraffe_328.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_033.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_320.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_007.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_263.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_388.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_378.png (deflated 1%)
  adding: content/cortical/img/giraffe/giraffe_283.png (deflated 1%)
  